<a href="https://colab.research.google.com/github/Sophie-X31/ECO372-Problem-Sets/blob/main/Problem_Set_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Import Statements
install.packages("haven")
library(haven)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
## Load Data File
url <- "https://raw.githubusercontent.com/Sophie-X31/ECO372-Problem-Sets/main/star.dta"
data <- read_dta(url)
head(data)

schidkn,pscore,classid,cs,female,nwhite,n
<dbl+lbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
22,0.00000000,6317,15,0,1,1
33,0.07998101,9700,26,1,1,1
56,0.03059830,16647,21,0,0,1
33,1.07327640,9700,26,0,1,1
45,0.35722697,13400,24,0,1,1
45,1.41278255,13400,24,0,1,1


In [18]:
## Inspect Data
# print(dim(data))
# print(colnames(data))
# print("Number of NA values in Columns:")
# na_columns <- colSums(is.na(data))
# print(na_columns[na_columns > 0])
data <- na.omit(data)

# Generate Dummy Variable
data$small <- ifelse(data$cs <= 17, 1, 0)

# Run OLS Analysis
ols_model <- lm(pscore ~ small, data = data)
print(summary(ols_model))

ols_model_b <- lm(pscore ~ small + female + nwhite, data = data)
print(summary(ols_model_b))


Call:
lm(formula = pscore ~ small, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-54.762 -22.347   0.981  22.669  49.721 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  50.0050     0.4319 115.776  < 2e-16 ***
small         4.7571     0.7626   6.238 4.74e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 26.97 on 5740 degrees of freedom
Multiple R-squared:  0.006734,	Adjusted R-squared:  0.006561 
F-statistic: 38.91 on 1 and 5740 DF,  p-value: 4.743e-10


Call:
lm(formula = pscore ~ small + female + nwhite, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-59.279 -21.298   0.717  21.590  58.877 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  51.1047     0.5890  86.764  < 2e-16 ***
small         4.6527     0.7446   6.248 4.45e-10 ***
female        5.3317     0.6956   7.665 2.09e-14 ***
nwhite      -11.2188     0.7414 -15.132  < 2e-16 

In [21]:
# Define the Four Strata
data$stratum <- interaction(data$female, data$nwhite, sep = "_")
data$stratum <- factor(data$stratum,
                       levels = c("0_0", "0_1", "1_0", "1_1"),
                       labels = c("Male_White", "Male_Nonwhite", "Female_White", "Female_Nonwhite"))

# Find Proportion
strata_proportions <- prop.table(table(data$stratum))
print(strata_proportions)

# Compute TE
strata_treatment_effects <- tapply(data$pscore, list(data$stratum, data$small), mean)
print(strata_treatment_effects)


     Male_White   Male_Nonwhite    Female_White Female_Nonwhite 
      0.3510972       0.1617903       0.3223615       0.1647510 
                       0        1
Male_White      51.17470 56.40480
Male_Nonwhite   38.72314 45.40843
Female_White    56.51422 60.02824
Female_Nonwhite 46.12584 49.72900


In [20]:
# Summarize Proportions
prop_female_small_1 <- mean(data$female[data$small == 1])
prop_female_small_0 <- mean(data$female[data$small == 0])
prop_nwhite_small_1 <- mean(data$nwhite[data$small == 1])
prop_nwhite_small_0 <- mean(data$nwhite[data$small == 0])

cat("Proportion of individuals with female = 1 among small = 1: ", prop_female_small_1, "\n")
cat("Proportion of individuals with female = 1 among small = 0: ", prop_female_small_0, "\n")
cat("Proportion of individuals with nwhite = 1 among small = 1: ", prop_nwhite_small_1, "\n")
cat("Proportion of individuals with nwhite = 1 among small = 0: ", prop_nwhite_small_0, "\n")

Proportion of individuals with female = 1 among small = 1:  0.4918567 
Proportion of individuals with female = 1 among small = 0:  0.4848718 
Proportion of individuals with nwhite = 1 among small = 1:  0.3224756 
Proportion of individuals with nwhite = 1 among small = 0:  0.3284615 
